In [2]:
import os

In [4]:
API_KEY = os.environ.get("API_KEY")
API_SECRET_KEY = os.environ.get("API_SECRET_KEY")
ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN")
ACCESS_TOKEN_SECRET = os.environ.get("ACCESS_TOKEN_SECRET")

In [3]:
def load_env():
    '''
    Load in Twitter API keys & tokens via os environment variables.
    '''
    API_KEY = os.environ.get("API_KEY")
    API_SECRET_KEY = os.environ.get("API_SECRET_KEY")
    ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN")
    ACCESS_TOKEN_SECRET = os.environ.get("ACCESS_TOKEN_SECRET")
    return API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

In [4]:
API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET = load_env()

In [5]:
import os
import csv
import pandas as pd
import tweepy
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
os.getcwd() + '/data'

'/Users/jai/Documents/Private-projects/Russian-Disinformation/data'

In [7]:
user = "earny_joe"

print(f"/data/{user}")

/data/earny_joe


In [54]:
def is_retweet(tweet):
    '''
    Returns True/False if Tweet is a retweet.
    '''
    if "RT @" in tweet.full_text:
        return True
    else:
        return False

In [55]:
from pathlib import Path

path = Path(os.getcwd())
print(path)

/Users/jai/Documents/Private-projects/Russian-Disinformation


In [56]:
path/f"data/{user}.csv"

PosixPath('/Users/jai/Documents/Private-projects/Russian-Disinformation/data/earny_joe.csv')

In [57]:
def get_tweets(path, user):
    '''
    Function that gathers input user's Tweets and outputs them to a CSV file.
    '''
    # open new CSV file into data folder of current directory
    csv_file = open(path/f"data/{user}.csv", "a")
    # create CSV writer
    csv_writer = csv.writer(csv_file)
    
    # write a single row with the headers of the columns
    csv_writer.writerow(
        [
            "id_str",
            "screen_name",
            "created_at",
            "lang",
            "source",
            "retweet_count",
            "favorite_count",
            "is_retweet",
            "full_text"
        ]
    )
    
    # Tweepy authorization
    auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
    
    # set Tweepy access token's
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    
    # call Twitter API
    api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    # get Tweets
    for tweet in tweepy.Cursor(api.user_timeline, screen_name=user, tweet_mode="extended").items():
        csv_writer.writerow(
            [
                tweet.id_str,
                tweet.user.screen_name,
                tweet.created_at,
                tweet.lang,
                tweet.source,
                tweet.retweet_count,
                tweet.favorite_count,
                is_retweet(tweet),
                tweet.full_text
            ]
        )
        
    # close csv file
    csv_file.close()

In [58]:
%%time

get_tweets(path, "earny_joe")

CPU times: user 1.01 s, sys: 76.8 ms, total: 1.08 s
Wall time: 14.7 s


In [59]:
df = pd.read_csv(path/'data/earny_joe.csv')

In [60]:
df.head()

,id_str,screen_name,created_at,lang,source,retweet_count,favorite_count,is_retweet,full_text
0,1208134986513010690,earny_joe,2019-12-20 21:20:01,en,Twitter for iPhone,351,0,True,RT @lexfridman: I often hear people say that X...
1,1208129576951398400,earny_joe,2019-12-20 20:58:31,en,Twitter Web App,0,1,False,Been wondering how to more securely store API ...
2,1208080971595075584,earny_joe,2019-12-20 17:45:23,en,Twitter Web App,15,0,True,RT @david_perell: The Internet rewards people ...
3,1208080911083851776,earny_joe,2019-12-20 17:45:08,en,Twitter Web App,114,0,True,"RT @RyanHoliday: ""Be satisfied with even the s..."
4,1208043688141692928,earny_joe,2019-12-20 15:17:14,en,Twitter for iPhone,3,0,True,RT @radekosmulski: ✅What is loneliness?\n✅What...


In [37]:
tweet_list = []
for tweet in tqdm(tweepy.Cursor(api.user_timeline, screen_name="earny_joe", tweet_mode="extended").items(5)):
    print(
        tweet.id_str,
        tweet.user.screen_name,
        tweet.created_at,
        tweet.lang,
        tweet.source,
        tweet.retweet_count,
        tweet.favorite_count,
        retweeted_status(tweet),
        tweet.user.verified,
        tweet.full_text,
        "\n"
    )

5it [00:00, 19.17it/s]

1208134986513010690 earny_joe 2019-12-20 21:20:01 en Twitter for iPhone 346 0 False False RT @lexfridman: I often hear people say that X is broken. Congress is broken. Academia is broken. Physics is broken. Deep learning is broke… 

1208129576951398400 earny_joe 2019-12-20 20:58:31 en Twitter Web App 0 1 True False Been wondering how to more securely store API keys/tokens, this article shows you how to do just that within @anacondainc  environments. 

Link: https://t.co/g8oePwX1bp 

1208080971595075584 earny_joe 2019-12-20 17:45:23 en Twitter Web App 15 0 False False RT @david_perell: The Internet rewards people who are prolific. 

Note-taking is the best way to become a prolific writer. 

1208080911083851776 earny_joe 2019-12-20 17:45:08 en Twitter Web App 114 0 False False RT @RyanHoliday: "Be satisfied with even the smallest progress, and treat the outcome of it all as unimportant." Marcus Aurelius 

1208043688141692928 earny_joe 2019-12-20 15:17:14 en Twitter for iPhone 3 0 False F

In [15]:
def tweepy_setup():
    '''
    Set's up Twitter API
    '''
    # Tweepy authorization
    auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
    
    # set Tweepy access token's
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    
    # call Twitter API
    api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except:
        print("Unverified credentials.")
    
    # return api
    return api

In [16]:
api = tweepy_setup()

In [17]:
tweet_list = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name="earny_joe", tweet_mode="extended").items(5):
    tweet_list.append(tweet._json)

In [20]:
# gather information regarding first tweet
first_tweet = tweet_list[0]

In [21]:
[key for key in first_tweet.keys()]

['created_at',
 'id',
 'id_str',
 'full_text',
 'truncated',
 'display_text_range',
 'entities',
 'source',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'user',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'retweeted_status',
 'is_quote_status',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'lang']

In [23]:
for tweet in tweet_list:
    if not "RT @" in tweet["full_text"]:
        try:
            print(tweet["full_text"])
        except:
            print("Exception.")

Been wondering how to more securely store API keys/tokens, this article shows you how to do just that within @anacondainc  environments. 

Link: https://t.co/g8oePwX1bp


In [33]:
def retweeted_status(status):
    '''
    Returns True/False if Tweet is a retweet.
    '''
    if not "RT @" in status.full_text:
        return True
    else:
        return False